In [1]:
from nfsp_agent_clean import NFSPDQN
from agent_configs import NFSPDQNConfig, RainbowConfig
from game_configs import MississippiMarblesConfig, LeducHoldemConfig, TicTacToeConfig
from utils import KLDivergenceLoss, CategoricalCrossentropyLoss, HuberLoss, MSELoss
from torch.optim import Adam, SGD

config_dict = {
    "shared_networks_and_buffers": True,
    "training_steps": 500000,  # like 2-5M in the paper (1M for initial test and see if rainbow is faster sooner)
    "num_players": 2,
    "anticipatory_param": 0.1,
    "replay_interval": 128,  # 128
    "num_minibatches": 2,  # 2 # 4
    "activation": "relu",
    "kernel_initializer": "he_normal",
    "learning_rate": 0.025,  # maybe should be lower for distributional, loss magnitudes are similar but there are many more weights for distributions (from atom size)? testing alpha/2 and ... # DQN / 4 # 0.1 for DQN so 0.025 for Rainbow (initial test used 0.1 for rainbow)
    "clipnorm": 1.0,
    "optimizer": Adam,  # SGD
    "loss_function": KLDivergenceLoss(),
    "per_alpha": 0.6,
    "per_beta": 0.5,
    "per_epsilon": 1e-3,
    "training_iterations": 1,  # 1
    "min_replay_buffer_size": 128,
    "minibatch_size": 128,
    "replay_buffer_size": 50000,  # 200000
    "transfer_interval": 300,  # 100
    "n_step": 3,  ##########
    "atom_size": 51,
    # "conv_layers": [(64, 3, 1), (32, 2, 1)],
    # "dense_layer_widths": [],
    # "value_hidden_layer_widths": [],
    # "advantage_hidden_layer_widths": [],
    "residual_layers": [(64, 3, 1), (64, 2, 1)],
    "conv_layers": [],
    "dense_layer_widths": [],
    "dueling": True,
    "value_hidden_layer_widths": [],
    "advantage_hidden_layer_widths": [],
    "noisy_sigma": 0.12,  ##########
    "eg_epsilon": 0.06,  # 0
    "eg_epsilon_decay_type": "inverse_sqrt",
    "eg_epsilon_decay_final_step": 0,
    "sl_activation": "relu",
    "sl_kernel_initializer": "he_normal",
    "sl_learning_rate": 0.005,  # 0.00005
    "sl_clipnorm": 1.0,
    "sl_optimizer": Adam,  # SGD
    "sl_loss_function": CategoricalCrossentropyLoss(),
    "sl_training_iterations": 1,  # 1
    "sl_min_replay_buffer_size": 128,
    "sl_minibatch_size": 128,
    "sl_replay_buffer_size": 1000000,  # 2000000
    # "sl_conv_layers": [(64, 3, 1), (32, 2, 1)],
    # "sl_dense_layer_widths": [],
    "sl_residual_layers": [],
    "sl_conv_layers": [(64, 3, 1)],
    "sl_dense_layer_widths": [],
}
# config = NFSPConfig(config_dict=config_dict, game_config=MississippiMarblesConfig(), rl_config_type=RainbowConfig)
config = NFSPDQNConfig(
    config_dict=config_dict,
    game_config=TicTacToeConfig(),
)
# config = NFSPDQNConfig(config_dict=config_dict, game_config=TicTacToeConfig(), rl_config_type=RainbowConfig)

Using default save_intermediate_weights     : False
Using         training_steps                : 500000
Using default adam_epsilon                  : 1e-06
Using         learning_rate                 : 0.025
Using         clipnorm                      : 1.0
Using         optimizer                     : <class 'torch.optim.adam.Adam'>
Using         loss_function                 : <utils.utils.KLDivergenceLoss object at 0x107e9bf10>
Using         activation                    : relu
Using         kernel_initializer            : he_normal
Using         minibatch_size                : 128
Using         replay_buffer_size            : 50000
Using         min_replay_buffer_size        : 128
Using         num_minibatches               : 2
Using         training_iterations           : 1
Using         num_players                   : 2
Using default save_intermediate_weights     : False
Using         training_steps                : 500000
Using default adam_epsilon                  : 1e-06
Usin

In [ ]:
from pathlib import Path

# TODO: 8, 9, 10, 11, 12
# DONE: 14, 13
dir = "./checkpoints/bad_test_func/NFSPDQN-LeducHoldem-8"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))

In [2]:
import custom_gym_envs
import gymnasium as gym

# env = gym.make('custom_gym_envs/MississippiMarbles-v0', render_mode="human", players=2)
env = gym.make(
    "custom_gym_envs/TicTacToe-v0", render_mode="rgb_array", encode_player_turn=True
)
# env = gym.make('custom_gym_envs/TicTacToe-v0', render_mode="rgb_array", player_turn_as_plane=False)
agent = NFSPDQN(env, config, name="NFSP-TicTacToe", device="cpu")

observation_dimensions:  (3, 3, 3)
num_actions:  9
observation_dimensions:  (3, 3, 3)
num_actions:  9


/opt/homebrew/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/dqn/NFSP folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


uint8
observation_dimensions:  (3, 3, 3)
num_actions:  9


/opt/homebrew/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.lz4_compress to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.lz4_compress` for environment variables or `env.get_wrapper_attr('lz4_compress')` that will search the reminding wrappers.
  logger.warn(
/opt/homebrew/lib/python3.10/site-packages/gymnasium/wrappers/record_video.py:94: UserWarning: WARN: Overwriting existing videos at /Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/dqn/NFSP folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


In [ ]:
dir = "./checkpoints/NFSPDQN-LeducHoldem"
agent.load_from_checkpoint(dir, 583000)

In [3]:
agent.checkpoint_trials = 100
agent.checkpoint_interval = 1000
agent.config.save_intermediate_weights = True
agent.train()

/opt/homebrew/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be uint8, actual type: float64
  logger.warn(
/opt/homebrew/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/opt/homebrew/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be uint8, actual type: float64
  logger.warn(
/opt/homebrew/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")
/opt/homebrew/lib/python3.10/site-packages/gymnasium/uti

Trial  0
Prediction tensor([[1.3496e-02, 6.9986e-05, 9.1757e-03, 5.6310e-02, 8.1065e-01, 5.1642e-05,
         4.1310e-02, 3.6889e-02, 3.2052e-02]])
7
Prediction tensor([[ 0.0107,  0.2108,  0.2137, -0.0182,  0.2938, -0.1658,  0.0941,    -inf,
          0.1936]])
4
Prediction tensor([[5.3325e-02, 3.1221e-02, 8.6449e-02, 1.2762e-01, 0.0000e+00, 3.3480e-05,
         6.5338e-01, 0.0000e+00, 4.7962e-02]])
6
Prediction tensor([[-0.0876,  0.1353,  0.1296, -0.1267,    -inf, -0.2457,    -inf,    -inf,
          0.1080]])
1
Prediction tensor([[0.0708, 0.0000, 0.0206, 0.5722, 0.0000, 0.1878, 0.0000, 0.0000, 0.1486]])
3
Prediction tensor([[-0.5047,    -inf,  0.6538,    -inf,    -inf, -0.1945,    -inf,    -inf,
         -0.2121]])
2
Prediction tensor([[0.0040, 0.0000, 0.0000, 0.0000, 0.0000, 0.9772, 0.0000, 0.0000, 0.0188]])
5
Prediction tensor([[-0.6274,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
         -0.0409]])
8
Prediction tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0.]])
0
T

Moviepy - Done !
Moviepy - video ready checkpoints/NFSP-TicTacToe/step_1000/videos/self.model_name_0/1000/NFSP-TicTacToe-episode-99.mp4
-0.81
Trial  0
Prediction tensor([[ 0.2014,  0.2617,  0.2620, -0.0804,  0.6800, -0.2504,  0.2837,  0.3905,
          0.5846]])
4
Prediction tensor([[0.0051, 0.0078, 0.0178, 0.4307, 0.0000, 0.0004, 0.1894, 0.3353, 0.0136]])
7
Prediction tensor([[ 0.3892,  0.3770,  0.2962,  0.0096,    -inf, -0.2109,  0.3470,    -inf,
          0.6024]])
8
Prediction tensor([[0.7312, 0.0464, 0.0419, 0.0925, 0.0000, 0.0096, 0.0784, 0.0000, 0.0000]])
0
Prediction tensor([[   -inf,  0.5539,  0.8723,  0.8435,    -inf,  0.8384, -0.1512,    -inf,
            -inf]])
2
Prediction tensor([[0.0000, 0.4253, 0.0000, 0.1941, 0.0000, 0.2050, 0.1757, 0.0000, 0.0000]])
6
Prediction tensor([[  -inf, 0.2504,   -inf, 0.7097,   -inf, 0.9711,   -inf,   -inf,   -inf]])
5
Trial  1
Prediction tensor([[ 0.2014,  0.2617,  0.2620, -0.0804,  0.6800, -0.2504,  0.2837,  0.3905,
          0.5846]])
4


3
Prediction tensor([[ 0.6109,  0.5238,  0.3789,    -inf,    -inf, -0.2077,  0.4620,  0.7478,
          0.7467]])
7
Prediction tensor([[0.0983, 0.5942, 0.0515, 0.0000, 0.0000, 0.1449, 0.0781, 0.0000, 0.0330]])
0
Prediction tensor([[   -inf,  0.9851, -0.0078,    -inf,    -inf, -0.1265,  0.4037,    -inf,
          0.5367]])
1
Trial  18
Prediction tensor([[ 0.2014,  0.2617,  0.2620, -0.0804,  0.6800, -0.2504,  0.2837,  0.3905,
          0.5846]])
4
Prediction tensor([[0.0051, 0.0078, 0.0178, 0.4307, 0.0000, 0.0004, 0.1894, 0.3353, 0.0136]])
3
Prediction tensor([[ 0.6109,  0.5238,  0.3789,    -inf,    -inf, -0.2077,  0.4620,  0.7478,
          0.7467]])
7
Prediction tensor([[0.0983, 0.5942, 0.0515, 0.0000, 0.0000, 0.1449, 0.0781, 0.0000, 0.0330]])
1
Prediction tensor([[ 0.8071,    -inf,  0.3379,    -inf,    -inf, -0.2083,  0.7367,    -inf,
          0.8496]])
8
Prediction tensor([[0.4355, 0.0000, 0.0376, 0.0000, 0.0000, 0.4955, 0.0314, 0.0000, 0.0000]])
5
Prediction tensor([[0.9959,   -inf

Moviepy - Done !
Moviepy - video ready checkpoints/NFSP-TicTacToe/step_1000/videos/self.model_name_1/1000/NFSP-TicTacToe-episode-199.mp4
0.84
0.029999999999999916


...

Moviepy - Done !
Moviepy - video ready checkpoints/NFSP-TicTacToe/step_8000/videos/self.model_name_0/8000/NFSP-TicTacToe-episode-1499.mp4
-0.22
Trial  0
Prediction tensor([[ 0.4376,  0.1822,  0.3807,  0.3842,  0.4548, -0.4363,  0.3192,  0.1108,
          0.2914]])
4
Prediction tensor([[4.6947e-01, 1.2932e-01, 1.1816e-01, 1.3127e-01, 0.0000e+00, 4.2203e-05,
         5.5464e-02, 7.0801e-02, 2.5478e-02]])
3
Prediction tensor([[ 0.5244,  0.6504,  0.3382,    -inf,    -inf, -0.1571,  0.7857,  0.6817,
          0.7347]])
6
Prediction tensor([[0.0582, 0.2437, 0.5521, 0.0000, 0.0000, 0.0118, 0.0000, 0.0664, 0.0678]])
2
Prediction tensor([[ 0.3388,  0.2128,    -inf,    -inf,    -inf, -0.0621,    -inf,  0.5959,
          0.2490]])
7
Prediction tensor([[0.3670, 0.2993, 0.0000, 0.0000, 0.0000, 0.1416, 0.0000, 0.0000, 0.1920]])
0
Prediction tensor([[  -inf, 0.9998,   -inf,   -inf,   -inf, 0.6644,   -inf,   -inf, 0.7990]])
1
Trial  1
Prediction tensor([[ 0.4376,  0.1822,  0.3807,  0.3842,  0.4548, -0

Moviepy - Done !
Moviepy - video ready checkpoints/NFSP-TicTacToe/step_8000/videos/self.model_name_1/8000/NFSP-TicTacToe-episode-1599.mp4
0.85
0.63


/Users/jonathanlamontange-kratz/Documents/GitHub/rl-stuff/packages/utils/utils/utils.py:278: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  axs[row][col].legend()


[{'exploitability': 0.029999999999999916}, {'exploitability': 0.7899999999999999}, {'exploitability': 1.34}, {'exploitability': 0.56}, {'exploitability': 0.5700000000000001}, {'exploitability': 0.7}, {'exploitability': -0.04000000000000001}, {'exploitability': 0.63}]


KeyboardInterrupt: 

In [ ]:
from pathlib import Path
from agent_configs import NFSPDQNConfig
from nfsp_agent_clean import NFSPDQN
import custom_gym_envs
import gymnasium as gym

# TODO: 8, 9, 10, 11, 12
# DONE: 14, 13
dir = "./checkpoints/Rainbow-TicTacToe"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))

env = gym.make(
    "custom_gym_envs/TicTacToe-v0", render_mode="human", encode_player_turn=True
)

agent = NFSPDQN(env, config, name="Rainbow-TicTacToe", device="cpu")

agent.load_from_checkpoint(dir, 20000)

In [ ]:
env = gym.make(
    "custom_gym_envs/TicTacToe-v0", render_mode="human", encode_player_turn=True
)
state, info = env.reset()
player = 0
agent_player = 1
done = False
agent.policies = ["best_response", "best_response"]
while not done:
    print(f"Player {player}")
    if player % 2 == agent_player:
        prediction = agent.predict(state, info)
        action = agent.select_actions(prediction, info).item()
    else:
        action = int(input("Enter action: "))
    state, reward, terminated, truncated, info = env.step(action)
    print(action)
    player = (player + 1) % 2
    env.render()
    done = terminated or truncated

In [ ]:
from pathlib import Path
import yaml
from utils import plot_comparisons
from agent_configs import NFSPDQNConfig
from nfsp_agent_clean import NFSPDQN
import gymnasium as gym
import custom_gym_envs

stats_list = []

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=False)

dir = "./checkpoints/NFSPDQN-LeducHoldem-7"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 525000)
stats_list.append(agent.stats)

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
dir = "./checkpoints/NFSPDQN-LeducHoldem-8"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 250000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-9"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 250000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-10"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 275000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-11"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 525000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-12"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 250000)
stats_list.append(agent.stats)

dir = "./checkpoints/NFSPDQN-LeducHoldem-13"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 85000)
stats_list.append(agent.stats)


dir = "./checkpoints/NFSPDQN-LeducHoldem"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
agent.load_from_checkpoint(dir, 685000)
stats_list.append(agent.stats)

In [ ]:
# indices:
# 0: Default
# 1: Default Shared
# 2: PER + Shared
# 3: Dueling + PER + Shared
# 4: Distributional + Dueling + PER + Shared
# 5: Distributional + Dueling + PER + Shared + LR 0.05
plot_comparisons(stats_list, "NFSPDQN-LeducHoldem")

In [ ]:
from agent_configs import NFSPDQNConfig
from nfsp_agent_clean import NFSPDQN
import gymnasium as gym
import custom_gym_envs
from pathlib import Path

# the test agent
env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
dir = "./checkpoints/NFSPDQN-LeducHoldem"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
test_agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
test_agent.load_from_checkpoint(dir, 750000)

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
dir = "./checkpoints/NFSPDQN-LeducHoldem"
config = NFSPDQNConfig.load(Path(dir, "configs/config.yaml"))
challenger_agent = NFSPDQN(env, config, name="NFSPDQN-LeducHoldem", device="cpu")
challenger_agent.load_from_checkpoint(dir, 750000)

test_agent.policies = ["average_strategy", "average_strategy"]
# the challenger agent
challenger_agent.policies = ["best_response", "best_response"]

In [ ]:
import copy


test_player = 0
score = 0
test_score = 0
env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
for _ in range(5000):
    print("Trial ", _)
    state, info = env.reset()
    done = False
    while not done:
        for player in range(2):
            if player == 0:
                prediction = test_agent.predict(state, info)
                action = test_agent.select_actions(prediction, info).item()
            else:
                prediction = challenger_agent.predict(state, info)
                action = challenger_agent.select_actions(prediction, info).item()
            print("Prediction", prediction)
            action_string = (
                "call"
                if action == 0
                else ("raise" if action == 1 else "fold" if action == 2 else "check")
            )
            print(action_string)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            state = next_state
            average_strategy_reward = reward[test_player]
            total_reward = sum(reward)
            test_score += total_reward - average_strategy_reward
            if done:
                break
score = test_score / 5000  #


test_player = 1
test_score = 0

env = gym.make("custom_gym_envs/LeducHoldem-v0", encode_player_turn=True)
for _ in range(5000):
    print("Trial ", _)
    state, info = env.reset()
    done = False
    while not done:
        for player in range(2):
            if player == 0:
                prediction = challenger_agent.predict(state, info)
                action = challenger_agent.select_actions(prediction, info).item()
            else:
                prediction = test_agent.predict(state, info)
                action = test_agent.select_actions(prediction, info).item()
            print("Prediction", prediction)
            action_string = (
                "call"
                if action == 0
                else ("raise" if action == 1 else "fold" if action == 2 else "check")
            )
            print(action_string)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            state = next_state
            average_strategy_reward = reward[test_player]
            total_reward = sum(reward)
            test_score += total_reward - average_strategy_reward
            if done:
                break
score += test_score / 5000  #
print(score)

In [ ]:
player_1 = agent.nfsp_agents[0]
player_2 = agent.nfsp_agents[1]
player_1.policy = "best_response"
state, info = env.reset()
print(state)
prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)


state, reward, terminated, truncated, info = env.step(4)
print(state)


prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)


state, reward, terminated, truncated, info = env.step(7)
print(state)


prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)


state, reward, terminated, truncated, info = env.step(3)
print(state)
print(terminated)


prediction = player_1.predict(state, info)
action = player_1.select_actions(prediction, info).item()
state, reward, terminated, truncated, info = env.step(action)
print(action)
print(state)

In [ ]:
state, info = env.reset()
print(state)
state_2, reward, terminated, truncated, info = env.step(0)
print(state_2)
print(state)

In [ ]:
samples = agent.nfsp_agents[0].rl_agent.replay_buffer.sample()
print(samples)

In [ ]:
import torch

q_values = torch.tensor(
    [
        [1, 0, 0, 0.5, -1],
        [-1, 1, 1, 1, -1],
    ]
)
legal_moves = [[0, 1, 3, 4], [2, 3, 4]]
mask = torch.zeros_like(q_values, dtype=torch.int8)
for i, legal in enumerate(legal_moves):
    mask[i, legal] = 1
print(mask)
q_values[mask == 0] = float("-inf")
selected_actions = q_values.argmax(1, keepdim=False)
print(q_values)